В этом домашнем задании вам необходимо будет использовать некоторый паттерн для того чтобы решить проблемы, описанные в условии. Для каждой задачи будет дан код, который должен работать после рефакторинга. В качестве решения напишите к какому паттерну проектирования вы будете приводить код и приведите отрефакторенный код.

### Задача 1: Динамическое управление шаблонами электронной почты (Decorator) – 0.5 б

#### Исходный код:
```python
class EmailTemplate:
    def render(self):
        return "Hello, User!"

template = EmailTemplate()
print(template.render())  # Hello, User!
```

#### Проблемы:
1. Добавление новых элементов (например, "Приветствие" или "Подпись") требует изменения основного класса.
2. Код становится негибким для динамических изменений.

#### Исправленный код:
```python
class EmailTemplate:
    def render(self):
        return "Hello, User!"

class HeaderDecorator:
    def __init__(self, template):
        self.template = template

    def render(self):
        return f"--- Header ---\n{self.template.render()}"

class FooterDecorator:
    def __init__(self, template):
        self.template = template

    def render(self):
        return f"{self.template.render()}\n--- Footer ---"

template = EmailTemplate()
decorated_template = FooterDecorator(HeaderDecorator(template))
print(decorated_template.render())
```

#### Обоснование:
- Decorator позволяет добавлять функциональность шаблона без изменения основного класса.
- Логика оформления становится модульной и легко расширяемой.

### Задача 2: Управление сценариями выполнения (Command) – 1 б

#### Исходный код:
```python
class TaskManager:
    def deploy(self):
        print("Deploying application...")

    def rollback(self):
        print("Rolling back deployment...")

manager = TaskManager()
action = input("Enter action (deploy/rollback): ")
if action == "deploy":
    manager.deploy()
elif action == "rollback":
    manager.rollback()
else:
    print("Unknown action")
```

#### Проблемы:
1. Каждый новый сценарий требует изменения `TaskManager`.
2. Невозможно логировать или отменять команды.

#### Исправленный код:
```python
class Command:
    def execute(self):
        pass

class DeployCommand(Command):
    def __init__(self, manager):
        self.manager = manager

    def execute(self):
        self.manager.deploy()

class RollbackCommand(Command):
    def __init__(self, manager):
        self.manager = manager

    def execute(self):
        self.manager.rollback()

class TaskManager:
    def deploy(self):
        print("Deploying application...")

    def rollback(self):
        print("Rolling back deployment...")

class CommandExecutor:
    def __init__(self):
        self.history = []

    def execute_command(self, command):
        command.execute()
        self.history.append(command)

manager = TaskManager()
executor = CommandExecutor()

deploy_command = DeployCommand(manager)
rollback_command = RollbackCommand(manager)

executor.execute_command(deploy_command)  # Deploying application...
executor.execute_command(rollback_command)  # Rolling back deployment...
```

#### Обоснование:
- Command изолирует действия, позволяя их логировать или комбинировать.
- Новые команды легко добавляются без изменения основного кода.

### Задача 3: Выбор метода авторизации (Strategy) – 1 б


#### Исходный код:
```python
def authenticate(auth_type, credentials):
    if auth_type == "basic":
        print(f"Authenticating with username and password: {credentials}")
    elif auth_type == "oauth":
        print(f"Authenticating with OAuth token: {credentials}")
    else:
        raise ValueError("Unknown authentication type")

authenticate("basic", {"username": "user", "password": "pass"})
authenticate("oauth", {"token": "abc123"})
```

#### Проблемы:
1. Новые методы авторизации требуют изменения функции.
2. Код трудно расширять и тестировать.

#### Исправленный код:
```python
class AuthStrategy:
    def authenticate(self, credentials):
        pass

class BasicAuthStrategy(AuthStrategy):
    def authenticate(self, credentials):
        print(f"Authenticating with username and password: {credentials}")

class OAuthStrategy(AuthStrategy):
    def authenticate(self, credentials):
        print(f"Authenticating with OAuth token: {credentials}")

class AuthContext:
    def __init__(self, strategy: AuthStrategy):
        self._strategy = strategy

    def set_strategy(self, strategy: AuthStrategy):
        self._strategy = strategy

    def authenticate(self, credentials):
        self._strategy.authenticate(credentials)

auth_context = AuthContext(BasicAuthStrategy())
auth_context.authenticate({"username": "user", "password": "pass"})

auth_context.set_strategy(OAuthStrategy())
auth_context.authenticate({"token": "abc123"})
```

#### Обоснование:
- Strategy изолирует алгоритмы авторизации, упрощая их добавление и тестирование.
- Код становится гибким и легко масштабируемым.

### Задача 4: Распределение задач между воркерами (Builder) – 0.5 б

#### Исходный код:
```python
class Worker:
    def __init__(self, tasks):
        self.tasks = tasks

worker = Worker(["task1", "task2", "task3"])
print(worker.tasks)
```

#### Проблемы:
1. Нет возможности гибко настраивать воркеров.
2. Список задач передаётся в конструктор, что затрудняет масштабирование.

#### Исправленный код:
```python
class Worker:
    def __init__(self):
        self.tasks = []

    def __str__(self):
        return f"Worker tasks: {', '.join(self.tasks)}"

class WorkerBuilder:
    def __init__(self):
        self.worker = Worker()

    def add_task(self, task):
        self.worker.tasks.append(task)
        return self

    def build(self):
        return self.worker

builder = WorkerBuilder()
worker = builder.add_task("task1").add_task("task2").add_task("task3").build()
print(worker)
```

#### Обоснование:
- Builder упрощает создание сложных объектов.
- Добавление новых задач становится декларативным и понятным.

### Задача 5: Контроль версии API с адаптерами (Adapter) – 0.5 б


#### Исходный код:
```python
class APIv1:
    def get_user(self):
        return {"id": 1, "name": "Alice"}

class APIv2:
    def get_user_data(self):
        return {"user": {"id": 1, "full_name": "Alice Johnson"}}

api = APIv1()
print(api.get_user())
```

#### Проблемы:
1. Переход на новую версию API потребует изменения всей логики вызовов.
2. Клиентский код не унифицирован для работы с разными версиями API.

#### Исправленный код:
```python
class APIv1:
    def get_user(self):
        return {"id": 1, "name": "Alice"}

class APIv2:
    def get_user_data(self):
        return {"user": {"id": 1, "full_name": "Alice Johnson"}}

class APIAdapter:
    def __init__(self, api):
        self.api = api

    def get_user(self):
        if hasattr(self.api, "get_user"):
            return self.api.get_user()
        elif hasattr(self.api, "get_user_data"):
            data = self.api.get_user_data()
            return {"id": data["user"]["id"], "name": data["user"]["full_name"]}

# Использование
api_v1 = APIAdapter(APIv1())
api_v2 = APIAdapter(APIv2())

print(api_v1.get_user())  # {'id': 1, 'name': 'Alice'}
print(api_v2.get_user())  # {'id': 1, 'name': 'Alice Johnson'}
```

#### Обоснование:
- Adapter позволяет унифицировать интерфейсы разных версий API.
- Клиентский код остаётся неизменным при переходе на новую версию.


### Задача 6: Контроль доступа к API (Proxy) – 0.5 б


#### Исходный код:
```python
class ExternalAPI:
    def fetch_data(self):
        print("Fetching data from external API...")
        return {"data": "external data"}

api = ExternalAPI()
print(api.fetch_data())
```

#### Проблемы:
1. Отсутствует контроль доступа: любой код может вызвать метод API.
2. Нет логирования запросов.

#### Исправленный код:
```python
class ExternalAPI:
    def fetch_data(self):
        print("Fetching data from external API...")
        return {"data": "external data"}

class APIProxy:
    def __init__(self, api, user_role):
        self.api = api
        self.user_role = user_role

    def fetch_data(self):
        if self.user_role != "admin":
            print("Access denied: Insufficient permissions")
            return None
        print("Logging: User accessed the API")
        return self.api.fetch_data()

# Использование
proxy = APIProxy(ExternalAPI(), user_role="user")
print(proxy.fetch_data())  # Access denied

admin_proxy = APIProxy(ExternalAPI(), user_role="admin")
print(admin_proxy.fetch_data())  # Logging + Fetching data
```

#### Обоснование:
- Proxy добавляет контроль доступа и логирование запросов.
- Логика доступа изолирована от оригинального API.
